In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import optuna
from tqdm.notebook import trange
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [ ]:
def read_data(ticker, offset = '0min'):
    """
    Read logarithmic returns from file

    :ticker: ticker of continious futures
    """
    # You need to insert the path to the data file
    data_folder = './'
    #All datasets for this course are available at this link:
    #https://drive.google.com/drive/folders/1mhbVjuwNZGX9nmZuCxJStBmraF3HIzio?usp=sharing

    if offset == '0min':
        data = pd.read_csv(data_folder+ticker+'.csv', index_col = 'date')
    else:
        data = pd.read_csv(data_folder+ticker+'_offset_'+offset+'.csv', index_col = 'date')

    # Parse timestamps
    data.index = pd.to_datetime(data.index, format = "%Y-%m-%d %H:%M:%S")

    return data

Functions for calculating standard statistics on the daily equity curve:

In [ ]:
days_in_year = 365.25

def Return(rets):
    """
    Annual return estimate

    :rets: daily returns of the strategy
    """
    return np.mean(rets)*days_in_year


def Volatility(rets):
    """
    Estimation of annual volatility

    :rets: daily returns of the strategy
    """
    return np.std(rets)*np.sqrt(days_in_year)


def SharpeRatio(rets):
    """
    Estimating the annual Sharpe ratio

    :rets: daily returns of the strategy
    """
    volatility = Volatility(rets)
    if (volatility>0):
        return Return(rets)/volatility
    else:
        return float('NaN')

def statistics_calc(rets, bh, name = '_', plot = False):
    """
    Draws a graph of portfolio equity and calculates annual Sharpe ratios, profitability and volatility

    :rets: daily returns of the strategy
    """
    sharpe = SharpeRatio(rets)
    ret = Return(rets)
    vol = Volatility(rets)
    if plot:
        plt.plot(rets.cumsum(), label = 'strategy')
        plt.xlabel('t')
    return  pd.DataFrame([[sharpe, ret, vol]], columns = ['Sharpe ratio', 'Annual return', 'Volatility'], index = [name])

In [ ]:
def strategy_backtest(data, params, plot = False, in_sample_end = '', slippage = 0.00001, plot_position = True):
    """
    Strategy backtest calculation
    Example call:
    strategy_bactest(test, [period, scale])

    :data: dataframe with log returns
    :params: list of strategy parameters
    :plot: if True than equity curve is plotted
    :in_sample_end: string in format "%Y-%m-%d" with timestamp of in_sample_end. Only used on charts
    :slippage: slippage per trade
    :return: statistics and equity curve
    """

    # Strategy parameters that we will optimize
    period = params[0]
    scale = params[1]

    signal_vol_period = params[2]
    vol_period_1 = params[3]
    vol_period_2 = params[4]
    pos_limit = params[5]
    reverse = params[6]

    # Calculation of target position:

    features = pd.DataFrame(index = data.index)

    # We calculate the exponential moving average of increments:
    features['signal'] = data['log_ret'].ewm(period).mean()

    # Estimating signal volatility using exponential moving average
    features['absSignal'] = np.abs(features['signal']).shift(1)
    features['signal_vol'] = features['absSignal'].ewm(signal_vol_period).mean()+0.000000001

    # Estimating asset volatility using double exponential moving average:
    features['absRet'] = np.abs(data['log_ret'])
    features['EmaAbsRet'] = features['absRet'].ewm(vol_period_1).mean().shift(1)*np.sqrt(vol_period_1)
    features['vol'] = features['EmaAbsRet'].ewm(vol_period_2).mean()*np.sqrt(vol_period_2)+0.000000001

    features['position'] = ((features['signal']/features['signal_vol']/features['vol']*scale).shift(1)).fillna(0).astype(int)

    # We remove looking into the future
    # (we cannot execute the order at the same price at which the signal was calculated. We need to take the price of the next bar)
    features['position'] = features['position'].shift(1)

    # We trade whole lots
    features['position'] = features['position'].fillna(0).astype(int)

    # Maximum position limit
    features.loc[features.index[features['position']>pos_limit], 'position'] = pos_limit
    features.loc[features.index[features['position']<-pos_limit], 'position'] = -pos_limit

    if reverse:
        features['position'] = -features['position']

    # We calculate the equity curve and convert it to a daily timeframe to calculate basic statistics
    eq = (data['log_ret']*features['position']-slippage*features['position'].diff().abs()
         ).fillna(0).resample('1D').agg('sum')
    bh = data['log_ret'].fillna(0).resample('1D').agg('sum')

    # We calculate statistics and save the result
    stats = statistics_calc(eq, bh, name = "{0}_{1}".format(period, scale), plot = plot)
    stats['period'] = period
    stats['scale'] = scale

    # Draw a graph of position changes, if necessary
    if (plot) and (plot_position):
        if in_sample_end  != '':
            plt.axvline(x = datetime.datetime.strptime(in_sample_end, "%Y-%m-%d").date(), color = 'red')
        plt.figure()
        position_to_plot = features['position'][-100000:]
        plt.plot(np.arange(len(position_to_plot)), position_to_plot)
        plt.title('position')
        plt.xlabel('t')

    return stats, eq

In [ ]:
in_sample_start = '2018-01-01'
in_sample_end = '2022-01-01'

data = read_data('NQ')
train = data[in_sample_start:in_sample_end]
test = data[in_sample_start:] # We intentionally included all the data in the test along with the training set to build a single equity curve

In [ ]:
def opt_backtest(period, scale, signal_vol_period, vol_period_1, vol_period_2, pos_limit, reverse):
    stats_current, _ = strategy_backtest(train, [period, scale, signal_vol_period, vol_period_1, vol_period_2, pos_limit, reverse])
    return stats_current['Sharpe ratio'].iloc[0]

In [ ]:
def objective(trial):

    period = trial.suggest_int("period", 10, 4000, log = True)
    scale = trial.suggest_float("scale", 0.001, 1.0, log = True)


    signal_vol_period = trial.suggest_int("signal_vol_period", 10, 2000, log = True)
    vol_period_1 = trial.suggest_int("vol_period_1", 10, 2000, log = True)
    vol_period_2 = trial.suggest_int("vol_period_2", 10, 2000, log = True)
    pos_limit = trial.suggest_int("pos_limit", 1, 6, log = True)

    reverse = trial.suggest_categorical("reverse", [True, False])

    obj_value = opt_backtest(period, scale, signal_vol_period, vol_period_1, vol_period_2, pos_limit, reverse)

    if np.isnan(obj_value):
        obj_value = 0

    return obj_value

In [ ]:
study = optuna.create_study(direction = 'maximize')
study.optimize(objective, n_trials = 1000, show_progress_bar = True, n_jobs = 1)
stats, eq = strategy_backtest(test, list(study.best_params.values()), True, in_sample_end)


In [ ]:
for i in trange(7):
    optimal_params = list(study.best_params.values())
    optimal_param_value = optimal_params[i]
    if (i != 1) and (i != 6):
        perturbated_param_values = np.unique((np.linspace(0.95*optimal_param_value, 1.05*optimal_param_value, 1000)).astype(int))
    if (i == 1):
        perturbated_param_values = ((np.linspace(0.95*optimal_param_value, 1.05*optimal_param_value, 1000)))
    if (i == 6):
        perturbated_param_values = [True, False]

    stats = pd.DataFrame()
    test_params = optimal_params
    for perturbated_param in (perturbated_param_values):
        test_params[i] = perturbated_param
        stats_current, _ = strategy_backtest(test, test_params, False, in_sample_end)
        stats_current.index = [str(perturbated_param)]
        if stats.shape[0] == 0:
            stats = stats_current
        else:
            stats = pd.concat([stats, stats_current])

    plt.figure()
    plt.hist(stats['Sharpe ratio'], bins = 100, density = True)

In [ ]:
tickers = ['BTC', 'ES', 'ETH', 'EU', 'GC', 'JY', 'NQ', 'YM']
offset_stats = pd.DataFrame(columns = ['Sharpe ratio', 'Mean(Sharpe ratio)', 'Std(Sharpe ratio)', 'Min(Sharpe ratio)'], index = tickers)

offset_stats_os = pd.DataFrame(columns = ['Sharpe ratio', 'Mean(Sharpe ratio)', 'Std(Sharpe ratio)', 'Min(Sharpe ratio)'], index = tickers)

for ticker in tickers:

    data = read_data(ticker)
    train = data[in_sample_start:in_sample_end]
    test = data[in_sample_start:] # We intentionally included all the data in the test along with the training set to build a single equity curve

    study = optuna.create_study(direction = 'maximize')
    study.optimize(objective, n_trials = 1000, show_progress_bar = True, n_jobs = 1)

    optimal_params = list(study.best_params.values())

    offsets = ['0min', '1min', '2min', '3min', '4min']
    stats = pd.DataFrame()
    for offset in offsets:
        data_off = read_data(ticker, offset)

        stats_current, _ = strategy_backtest(data_off, optimal_params, False, plot_position = False)
        stats_current.index = [offset]
        if stats.shape[0] == 0:
            stats = stats_current
        else:
            stats = pd.concat([stats, stats_current])

    offset_stats.loc[ticker, :] = [stats['Sharpe ratio'].iloc[0], stats['Sharpe ratio'].mean(), stats['Sharpe ratio'].std(), stats['Sharpe ratio'].min()]


    stats_os = pd.DataFrame()
    for offset in offsets:
        data_off = read_data(ticker, offset)

        stats_current, _ = strategy_backtest(data_off[in_sample_end:], optimal_params, False, plot_position = False)
        stats_current.index = [offset]
        if stats_os.shape[0] == 0:
            stats_os = stats_current
        else:
            stats_os = pd.concat([stats_os, stats_current])

    offset_stats_os.loc[ticker, :] = [stats_os['Sharpe ratio'].iloc[0], stats_os['Sharpe ratio'].mean(), stats_os['Sharpe ratio'].std(), stats_os['Sharpe ratio'].min()]


In [ ]:
offset_stats

In [ ]:
offset_stats_os

In [ ]:
plt.rc('axes', axisbelow = True)
plt.grid()
plt.bar(offset_stats.index, offset_stats['Sharpe ratio'], width = 0.2)

plt.errorbar(offset_stats.index, offset_stats['Mean(Sharpe ratio)'], yerr = offset_stats['Std(Sharpe ratio)'], fmt = 'o', color = 'tab:orange')

In [ ]:
plt.rc('axes', axisbelow = True)
plt.grid()
plt.bar(offset_stats_os.index, offset_stats_os['Sharpe ratio'], width = 0.2)

plt.errorbar(offset_stats_os.index, offset_stats_os['Mean(Sharpe ratio)'], yerr = offset_stats_os['Std(Sharpe ratio)'], fmt = 'o', color = 'tab:orange')